In [1]:
!pip install transformers datasets pandas sklearn
!pip install --upgrade transformers

  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## Needed Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import tensorflow as tf
from transformers import DistilBertTokenizerFast, GPT2Tokenizer, GPT2LMHeadModel, TFGPT2ForSequenceClassification
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer

## Reading and Preprocessing the Data

In [3]:
df = pd.read_csv("Emotion_classify_Data.csv")

In [4]:
print(df.shape)
df.head()

(5937, 2)


,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [5]:
df.isnull().sum()

,0
Comment,0
Emotion,0


In [6]:
print(f'# duplicate rows: {df.duplicated().sum()}')

# duplicate rows: 0


**You may use LabelEncoder() function or Map labels manually**

In [7]:
le = LabelEncoder()

df['Emotion'] = le.fit_transform(df['Emotion'])
df.head()

,Comment,Emotion
0,i seriously hate one subject to death but now ...,1
1,im so full of life i feel appalled,0
2,i sit here to write i start to dig out my feel...,1
3,ive been really angry with r and i feel like a...,2
4,i feel suspicious if there is no one outside l...,1


In [8]:
label_mapping = {'joy' : 0, 'fear': 1, 'anger': 2}
df['Emotion'] = df['Emotion'].map(label_mapping)
#df.drop('Emotion', axis=1, inplace=True)
df.head()

,Comment,Emotion
0,i seriously hate one subject to death but now ...,NaN
1,im so full of life i feel appalled,NaN
2,i sit here to write i start to dig out my feel...,NaN
3,ive been really angry with r and i feel like a...,NaN
4,i feel suspicious if there is no one outside l...,NaN


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['Comment'], df['Emotion'], test_size = 0.2, random_state = 42, stratify = df['Emotion'])

In [9]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (4749,)
Shape of X_test:  (1188,)


In [11]:
# Define a custom loss function
def custom_loss(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

### BERT classification head

In [12]:
# Set up distribution strategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy()

# Load and compile the model within the strategy scope
with strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=custom_loss,  # Use custom loss function
        metrics=['accuracy']
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [13]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
X_train = list(X_train)
X_test = list(X_test)

train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(8)

In [16]:
model.fit(train_dataset, epochs=8, validation_data=test_dataset)

Epoch 1/8
594/594 [==============================] - 91s 102ms/step - loss: 0.3200 - accuracy: 0.8690 - val_loss: 0.1126 - val_accuracy: 0.9588
Epoch 2/8
594/594 [==============================] - 56s 94ms/step - loss: 0.0832 - accuracy: 0.9680 - val_loss: 0.0732 - val_accuracy: 0.9739
Epoch 3/8
594/594 [==============================] - 57s 96ms/step - loss: 0.0700 - accuracy: 0.9718 - val_loss: 0.1093 - val_accuracy: 0.9663
Epoch 4/8
594/594 [==============================] - 55s 93ms/step - loss: 0.0451 - accuracy: 0.9834 - val_loss: 0.1060 - val_accuracy: 0.9663
Epoch 5/8
594/594 [==============================] - 56s 94ms/step - loss: 0.0296 - accuracy: 0.9901 - val_loss: 0.1464 - val_accuracy: 0.9689
Epoch 6/8
594/594 [==============================] - 57s 95ms/step - loss: 0.0309 - accuracy: 0.9907 - val_loss: 0.1049 - val_accuracy: 0.9739
Epoch 7/8
594/594 [==============================] - 56s 94ms/step - loss: 0.0294 - accuracy: 0.9924 - val_loss: 0.1867 - val_accuracy: 0.956

In [17]:
model.evaluate(test_dataset)

149/149 [==============================] - 4s 27ms/step - loss: 0.1271 - accuracy: 0.9731


[0.1270744800567627, 0.9730639457702637]

In [18]:
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=-1).numpy()


149/149 [==============================] - 6s 29ms/step


In [19]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predicted_labels)
print(f"Confusion Matrix:\n{cm}")


Confusion Matrix:
[[385   9   6]
 [ 12 375   1]
 [  3   1 396]]


In [20]:
model.save_pretrained('senti_model')
print("Model saved to 'senti_model'")

Model saved to 'senti_model'
